In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Set, Dict
from enum import Enum
import time

print("Libraries imported successfully!")

## Define Constants and Antenna Specifications

We define three antenna types with their properties:
- **Small**: Shortest range, cheapest
- **Medium**: Moderate range and cost
- **Large**: Longest range, most expensive

In [ ]:
# Constants
USERS_PER_HOUSE = 100

# Antenna Types
class AntennaType(str, Enum):
    SMALL = "Small"
    MEDIUM = "Medium"
    LARGE = "Large"

# Antenna Specifications
class AntennaSpec:
    def __init__(self, type: AntennaType, radius: int, cost: int):
        self.type = type
        self.radius = radius
        self.cost = cost
    
    def __repr__(self):
        return f"{self.type.value}(radius={self.radius}, cost=${self.cost})"

# Define antenna specifications
ANTENNA_SPECS = {
    AntennaType.SMALL: AntennaSpec(AntennaType.SMALL, radius=2, cost=1000),
    AntennaType.MEDIUM: AntennaSpec(AntennaType.MEDIUM, radius=4, cost=1500),
    AntennaType.LARGE: AntennaSpec(AntennaType.LARGE, radius=6, cost=2000),
}

print("Antenna Specifications:")
for spec in ANTENNA_SPECS.values():
    print(f"  {spec}")

## Implement Greedy Algorithm Class

The `GreedyAlgorithm` class implements a cost-optimized approach:
1. Evaluates all possible antenna placements
2. Selects the one with lowest cost per new house covered
3. Repeats until 100% coverage is achieved

In [ ]:
class GreedyAlgorithm:
    """Simplified greedy algorithm for cost-optimized antenna placement."""
    
    def __init__(
        self,
        width: int,
        height: int,
        antenna_specs: Dict[AntennaType, AntennaSpec],
        houses: List[Tuple[int, int]]
    ):
        """
        Initialize the greedy algorithm.
        
        Args:
            width: Grid width
            height: Grid height
            antenna_specs: Dictionary of antenna specifications
            houses: List of house coordinates (each has 100 users)
        """
        self.width = width
        self.height = height
        self.antenna_specs = antenna_specs
        self.houses = set(houses)
        self.covered_houses: Set[Tuple[int, int]] = set()
        self.placed_antennas: List[Dict] = []
        
        print(f"Initialized: {width}x{height} grid, {len(houses)} houses, {len(houses) * USERS_PER_HOUSE} total users")
    
    def get_covered_houses(self, x: int, y: int, radius: int) -> Set[Tuple[int, int]]:
        """Calculate which houses are covered by an antenna at position (x, y)."""
        covered = set()
        
        for house_x, house_y in self.houses:
            # Euclidean distance
            distance = ((x - house_x) ** 2 + (y - house_y) ** 2) ** 0.5
            if distance <= radius:
                covered.add((house_x, house_y))
        
        return covered
    
    def is_valid_position(self, x: int, y: int) -> bool:
        """Check if a position is valid for antenna placement (not on a house)."""
        return (0 <= x < self.width and 
                0 <= y < self.height and 
                (x, y) not in self.houses)
    
    def find_best_antenna_placement(self):
        """Find the best antenna placement with lowest cost per new house covered."""
        best_position = None
        best_antenna_type = None
        best_cost_efficiency = float('inf')  # Lower is better
        best_new_houses = set()
        
        # Try all antenna types
        for antenna_type, spec in self.antenna_specs.items():
            # Try all positions on the grid
            for x in range(self.width):
                for y in range(self.height):
                    if not self.is_valid_position(x, y):
                        continue
                    
                    # Calculate coverage
                    covered = self.get_covered_houses(x, y, spec.radius)
                    new_houses = covered - self.covered_houses
                    
                    if len(new_houses) > 0:
                        # Cost efficiency = cost per new house covered
                        cost_efficiency = spec.cost / len(new_houses)
                        
                        if cost_efficiency < best_cost_efficiency:
                            best_cost_efficiency = cost_efficiency
                            best_position = (x, y)
                            best_antenna_type = antenna_type
                            best_new_houses = new_houses
        
        if best_position is None:
            return None
        
        return best_position, best_antenna_type, best_new_houses
    
    def optimize(self) -> Dict:
        """Run the greedy algorithm to minimize cost while covering all houses."""
        print("\nStarting greedy optimization...")
        
        start_time = time.time()
        total_houses = len(self.houses)
        iteration = 0
        
        # Continue until all houses are covered
        while len(self.covered_houses) < total_houses:
            iteration += 1
            
            # Find best antenna placement
            result = self.find_best_antenna_placement()
            
            if result is None:
                print(f"\n⚠ Warning: Could not cover all houses. {len(self.covered_houses)}/{total_houses} covered.")
                break
            
            position, antenna_type, new_houses = result
            spec = self.antenna_specs[antenna_type]
            
            # Place antenna
            antenna_data = {
                "x": position[0],
                "y": position[1],
                "type": antenna_type,
                "radius": spec.radius,
                "cost": spec.cost
            }
            self.placed_antennas.append(antenna_data)
            
            # Update coverage
            self.covered_houses.update(new_houses)
            
            coverage_percent = (len(self.covered_houses) / total_houses * 100)
            
            print(f"  Iteration {iteration}: Placed {antenna_type.value} at {position}, "
                  f"covered {len(new_houses)} new houses, total coverage: {coverage_percent:.1f}%")
        
        # Calculate final statistics
        total_users = total_houses * USERS_PER_HOUSE
        houses_covered = len(self.covered_houses)
        users_covered = houses_covered * USERS_PER_HOUSE
        coverage_percentage = (users_covered / total_users * 100) if total_users > 0 else 0
        total_cost = sum(ant["cost"] for ant in self.placed_antennas)
        execution_time_ms = (time.time() - start_time) * 1000
        
        print(f"\n✓ Optimization complete!")
        print(f"  Antennas placed: {len(self.placed_antennas)}")
        print(f"  Total cost: ${total_cost:,}")
        print(f"  Houses covered: {houses_covered}/{total_houses}")
        print(f"  Users covered: {users_covered:,}/{total_users:,} ({coverage_percentage:.1f}%)")
        print(f"  Execution time: {execution_time_ms:.2f} ms")
        
        return {
            "antennas": self.placed_antennas,
            "houses_covered": houses_covered,
            "total_houses": total_houses,
            "users_covered": users_covered,
            "total_users": total_users,
            "coverage_percentage": coverage_percentage,
            "total_cost": total_cost,
            "execution_time_ms": execution_time_ms
        }

print("GreedyAlgorithm class defined successfully!")

## Create Sample Grid

Let's create a sample grid with random house placements and run the optimization.

In [ ]:
# Create a sample grid
np.random.seed(42)
GRID_WIDTH = 20
GRID_HEIGHT = 15

# Generate random house positions (15% of grid cells)
num_houses = int(GRID_WIDTH * GRID_HEIGHT * 0.15)
houses = []
while len(houses) < num_houses:
    x = np.random.randint(0, GRID_WIDTH)
    y = np.random.randint(0, GRID_HEIGHT)
    if (x, y) not in houses:
        houses.append((x, y))

print(f"Grid size: {GRID_WIDTH}x{GRID_HEIGHT}")
print(f"Number of houses: {len(houses)}")
print(f"Total users: {len(houses) * USERS_PER_HOUSE}")

## Run Greedy Algorithm

In [ ]:
# Initialize and run the greedy algorithm
algorithm = GreedyAlgorithm(
    width=GRID_WIDTH,
    height=GRID_HEIGHT,
    antenna_specs=ANTENNA_SPECS,
    houses=houses
)

result = algorithm.optimize()

## Visualize Results

Create a visualization showing:
- Houses (red squares)
- Antennas (colored by type)
- Coverage areas (semi-transparent circles)

In [ ]:
# Visualization
fig, ax = plt.subplots(1, 1, figsize=(14, 10))

# Define colors for each antenna type
antenna_colors = {
    AntennaType.SMALL: '#10B981',   # Green
    AntennaType.MEDIUM: '#3B82F6',  # Blue
    AntennaType.LARGE: '#8B5CF6',   # Purple
}

# Plot coverage circles first (so they're in the background)
for antenna in result['antennas']:
    color = antenna_colors[antenna['type']]
    circle = plt.Circle((antenna['x'], antenna['y']), antenna['radius'], 
                        color=color, alpha=0.15, zorder=1)
    ax.add_patch(circle)

# Plot houses
if houses:
    houses_x, houses_y = zip(*houses)
    ax.scatter(houses_x, houses_y, c='red', s=150, marker='s', 
              label='Houses (100 users each)', zorder=3, edgecolors='darkred', linewidth=2)

# Plot antennas
antenna_types = {}
for antenna in result['antennas']:
    ant_type = antenna['type']
    if ant_type not in antenna_types:
        antenna_types[ant_type] = {'x': [], 'y': []}
    antenna_types[ant_type]['x'].append(antenna['x'])
    antenna_types[ant_type]['y'].append(antenna['y'])

for ant_type, coords in antenna_types.items():
    color = antenna_colors[ant_type]
    ax.scatter(coords['x'], coords['y'], c=color, s=250, marker='^', 
              label=f'{ant_type.value} Antenna', zorder=4, 
              edgecolors='black', linewidth=2)

# Set grid
ax.set_xlim(-1, GRID_WIDTH)
ax.set_ylim(-1, GRID_HEIGHT)
ax.set_aspect('equal')
ax.grid(True, alpha=0.3)
ax.set_xlabel('X Coordinate', fontsize=12)
ax.set_ylabel('Y Coordinate', fontsize=12)
ax.set_title(f'Greedy Algorithm - Antenna Placement Optimization\nTotal Cost: ${result["total_cost"]:,}', 
            fontsize=14, fontweight='bold')

# Add legend
ax.legend(loc='upper left', bbox_to_anchor=(1, 1), fontsize=10)

# Add statistics text box
stats_text = f"""Statistics:
Antennas: {len(result['antennas'])}
Total Cost: ${result['total_cost']:,}
Houses Covered: {result['houses_covered']}/{result['total_houses']}
Users Covered: {result['users_covered']:,}
Coverage: {result['coverage_percentage']:.1f}%
Time: {result['execution_time_ms']:.2f} ms
"""
props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
ax.text(0.02, 0.98, stats_text, transform=ax.transAxes, fontsize=10,
        verticalalignment='top', bbox=props)

plt.tight_layout()
plt.show()

# Print antenna breakdown
print("\nAntenna Breakdown by Type:")
antenna_counts = {}
antenna_costs = {}
for antenna in result['antennas']:
    ant_type = antenna['type']
    antenna_counts[ant_type] = antenna_counts.get(ant_type, 0) + 1
    antenna_costs[ant_type] = antenna_costs.get(ant_type, 0) + antenna['cost']

for ant_type in sorted(antenna_counts.keys(), key=lambda x: x.value):
    count = antenna_counts[ant_type]
    cost = antenna_costs[ant_type]
    print(f"  {ant_type.value}: {count} antenna(s) - ${cost:,}")

## Test with Different Grid Sizes

Let's test how the algorithm performs on different grid sizes and densities.

In [ ]:
# Test with different densities
test_configs = [
    {"width": 15, "height": 15, "density": 0.10, "name": "Low Density (10%)"},
    {"width": 20, "height": 15, "density": 0.15, "name": "Medium Density (15%)"},
    {"width": 25, "height": 20, "density": 0.20, "name": "High Density (20%)"},
]

results_comparison = []

for config in test_configs:
    print("\n" + "="*70)
    print(f"TEST: {config['name']} - {config['width']}x{config['height']} grid")
    print("="*70)
    
    # Generate houses
    np.random.seed(42)
    num_houses = int(config['width'] * config['height'] * config['density'])
    test_houses = []
    while len(test_houses) < num_houses:
        x = np.random.randint(0, config['width'])
        y = np.random.randint(0, config['height'])
        if (x, y) not in test_houses:
            test_houses.append((x, y))
    
    # Run optimization
    algorithm = GreedyAlgorithm(
        width=config['width'],
        height=config['height'],
        antenna_specs=ANTENNA_SPECS,
        houses=test_houses
    )
    
    result = algorithm.optimize()
    results_comparison.append({
        "name": config['name'],
        "houses": num_houses,
        "users": result['total_users'],
        "antennas": len(result['antennas']),
        "cost": result['total_cost'],
        "cost_per_user": result['total_cost'] / result['total_users'] if result['total_users'] > 0 else 0
    })

# Display comparison
print("\n" + "="*70)
print("COMPARISON SUMMARY")
print("="*70)
print(f"{'Scenario':<25} {'Houses':<10} {'Users':<10} {'Antennas':<12} {'Cost':<12} {'$/User':<10}")
print("-" * 70)
for r in results_comparison:
    print(f"{r['name']:<25} {r['houses']:<10} {r['users']:<10,} {r['antennas']:<12} ${r['cost']:<11,} ${r['cost_per_user']:.2f}")

print("\nKey Insights:")
print(f"  - Cost per user ranges from ${min(r['cost_per_user'] for r in results_comparison):.2f} to ${max(r['cost_per_user'] for r in results_comparison):.2f}")
print(f"  - Higher density areas may be more cost-efficient per user")

## Algorithm Summary

**Key Features:**
- **Greedy approach**: Selects antenna placement with best cost per house covered
- **Constraint**: Antennas cannot be placed on houses
- **Goal**: 100% coverage with minimum cost
- **Complexity**: O(T × W × H × N) per iteration
  - T = 3 antenna types
  - W × H = grid size
  - N = number of houses
  
**How it works:**
1. For every empty position on the grid
2. Try all 3 antenna types
3. Calculate how many new houses would be covered
4. Select the antenna with lowest cost per new house
5. Repeat until all houses are covered

**Advantages:**
- ✓ Fast execution (milliseconds)
- ✓ Guaranteed 100% coverage
- ✓ Simple and predictable
- ✓ Good cost efficiency

**Limitations:**
- ✗ Locally optimal (may not find global optimum)
- ✗ No backtracking once antenna is placed
- ✗ Cannot explore alternative solutions